### Imports

In [81]:
import json
import networkx as nx

### JSON

In [82]:
with open('NFA_mt.json') as _:
    NFA = json.load(_)

Graph = nx.Graph()
#For checking purposes
NFA

{'startingState': 'S7',
 'S7': {'isTerminatingState': False, 'epsilon': ['S5', 'S8']},
 'S8': {'isTerminatingState': False, 'epsilon': 'S9'},
 'S5': {'isTerminatingState': False, 'epsilon': ['S3', 'S1']},
 'S6': {'isTerminatingState': False, 'epsilon': ['S7', 'S8']},
 'S3': {'isTerminatingState': False, 'b': 'S4'},
 'S4': {'isTerminatingState': False, 'epsilon': 'S6'},
 'S1': {'isTerminatingState': False, 'a': 'S2'},
 'S2': {'isTerminatingState': False, 'epsilon': 'S6'},
 'S9': {'isTerminatingState': False, 'a': 'S10'},
 'S10': {'isTerminatingState': False, 'epsilon': 'S17'},
 'S17': {'isTerminatingState': False, 'epsilon': ['S15', 'S18']},
 'S18': {'isTerminatingState': True},
 'S15': {'isTerminatingState': False, 'epsilon': ['S13', 'S11']},
 'S16': {'isTerminatingState': False, 'epsilon': 'S18'},
 'S13': {'isTerminatingState': False, 'b': 'S14'},
 'S14': {'isTerminatingState': False, 'epsilon': 'S16'},
 'S11': {'isTerminatingState': False, 'a': 'S12'},
 'S12': {'isTerminatingState': 

In [83]:
startingState = NFA['startingState']
keys = list(NFA.keys())
ins =  set()
DFA = {}

#For checking purposes
print(startingState)

S7


In [84]:
keys = list(NFA[startingState].keys())
for key in keys[1:]:
    ins.add(key)


In [85]:
def add_epsilon_moves(queue: list) -> list:
    visited = []

    while queue:
        state = queue.pop(0) 
        visited.append(state)
        if 'epsilon' in NFA[state]:
            epsilon_value = NFA[state]['epsilon']
            if isinstance(epsilon_value, list):
                for item in epsilon_value:
                    if item not in visited and item not in queue:
                        queue.append(item)
            else:
                if epsilon_value not in visited and epsilon_value not in queue:
                    queue.append(epsilon_value)
    return visited

In [86]:
def get_possible_inputs(states: set) -> set:
    inputs = set()
    for state in states:
        keys = list(NFA[state].keys())
        for key in keys[1:]:
            if key != 'epsilon' and key not in inputs:
                inputs.add(key)
    return inputs

In [87]:
def add_inputs(states: set, inputs: set) -> dict :
    state_inputs = {}
    for input in inputs:
        ns = set()
        for state in states:
            if(input in NFA[state]):
                ns.add(NFA[state][input])
        ns.update(add_epsilon_moves(list(ns)))
        state_inputs[input] = ns
    return state_inputs
        

In [88]:
def checkTerminating(states: set) -> bool:
    for state in states:
        if NFA[state]['isTerminatingState']:
            return True
    return False

In [89]:
states = set(); states_dict = {}; ini = True
states.add(startingState)
ct = 0; nname = f"NS{ct}"
states.update(add_epsilon_moves([startingState]))
inputs = get_possible_inputs(states)
outStates = add_inputs(states, inputs)
states_dict['startingState'] = nname
states_dict[nname] = {'isTerminatingState': checkTerminating(states)}; 
states_dict[nname]['states'] = states; ct+=1

states_dict

{'startingState': 'NS0',
 'NS0': {'isTerminatingState': False,
  'states': {'S1', 'S3', 'S5', 'S7', 'S8', 'S9'}}}

In [90]:
kv = list(states_dict.keys())
i = 0
while i < len(kv):
    ns = kv[i]
    ep_states = set().clear()
    if ns == "startingState":
        i += 1
        continue
    ep_states = add_epsilon_moves(list(states_dict[ns].get('states')))
    inputs = get_possible_inputs(ep_states)
    outStates = add_inputs(ep_states, inputs)
    for input in inputs:
        for key in list(states_dict.keys())[1:]:
            if(states_dict[key].get('states') == outStates[input]):
                nname = key
                break
        else:   #Only enter if not breaks
            nname = f"NS{ct}"
            states_dict[nname] = {'isTerminatingState': checkTerminating(outStates[input])}
            states_dict[nname]['states'] = outStates[input]
            ct+=1
        states_dict[ns][input] = nname
    kv = list(states_dict.keys())
    i += 1

states_dict

{'startingState': 'NS0',
 'NS0': {'isTerminatingState': False,
  'states': {'S1', 'S3', 'S5', 'S7', 'S8', 'S9'},
  'a': 'NS1',
  'b': 'NS2'},
 'NS1': {'isTerminatingState': True,
  'states': {'S1',
   'S10',
   'S11',
   'S13',
   'S15',
   'S17',
   'S18',
   'S2',
   'S3',
   'S5',
   'S6',
   'S7',
   'S8',
   'S9'},
  'a': 'NS3',
  'b': 'NS4'},
 'NS2': {'isTerminatingState': False,
  'states': {'S1', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9'},
  'a': 'NS1',
  'b': 'NS2'},
 'NS3': {'isTerminatingState': True,
  'states': {'S1',
   'S10',
   'S11',
   'S12',
   'S13',
   'S15',
   'S16',
   'S17',
   'S18',
   'S2',
   'S3',
   'S5',
   'S6',
   'S7',
   'S8',
   'S9'},
  'a': 'NS3',
  'b': 'NS4'},
 'NS4': {'isTerminatingState': True,
  'states': {'S1',
   'S14',
   'S16',
   'S18',
   'S3',
   'S4',
   'S5',
   'S6',
   'S7',
   'S8',
   'S9'},
  'a': 'NS1',
  'b': 'NS2'}}

In [91]:
############# Law Omar Samir Wafe2 3ala networkX ################
Graph = nx.DiGraph()
states = set()
ct = 1
states.add(startingState)
states.update(add_epsilon_moves([startingState]))
Graph.add_node('S0', states=states)
inputs = get_possible_inputs(states)
outStates = add_inputs(states, inputs)
for input in inputs:
    nname = f"S{ct}"
    Graph.add_node(nname, states = outStates[input])
    Graph.add_edge('S0',nname, input = input)
    ct+=1
print(Graph.nodes)
print(Graph.edges('S0'))

['S0', 'S1', 'S2']
[('S0', 'S1'), ('S0', 'S2')]
